In [1]:
from functools import reduce
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [2]:
All = pd.read_csv('data\All.csv')

In [3]:
# main dataset
price = pd.read_csv('data\Raw_Price.csv')

# feature dataset
## income
income = pd.read_excel('data\Income.xls') # long feature
income_inequality = pd.read_excel('data\Income_Inequality_Ratio.xls') # long feature
ad_household_wage = pd.read_excel('data\Mean_Household_Wages_Adjusted_by_Cost.xls')

## education
BD = pd.read_excel('data\Bachelor_Degree_or_Higher.xls') # long feature
high_school = pd.read_excel('data\High_School_Graduate_or_Higher.xls') # long feature

## market (monthly)
inventory_new = pd.read_excel('data\Inventory_New_Listing_Percent.xls')
inventory_total = pd.read_excel('data\Inventory_Total_Listing_Percent.xls')
demand = pd.read_excel('data\Market_Demand_Score.xls')
nielsen_rank = pd.read_excel('data\Market_Nielsen_Household_Rank.xls')
supply = pd.read_excel('data\Market_Supply_Score.xls')

## population
migration = pd.read_excel(r'data\Net_Migration_Flow.xls')  # long feature
resident = pd.read_excel('data\Resident_Population.xls')  # long feature


# # Get all data file names in folder "data"
# import os
# filePath = 'data\..'
# for dirpath, dirnames, filenames in os.walk(filePath):
#     print(filenames)

In [4]:
features = [income, income_inequality, ad_household_wage, BD, high_school, inventory_new, 
            inventory_total, demand, nielsen_rank, supply, migration, resident]
long_features = [income, income_inequality, BD, high_school, migration, resident]
short_features = [ad_household_wage, inventory_new, inventory_total, demand, nielsen_rank, supply]

In [5]:
# convert feature date columns into rows
for i in range(len(long_features)):
    print(long_features[i].shape)
    long_features[i] = long_features[i].rename(columns={'RegionName': 'County'})
    long_features[i]['County'] = long_features[i]['County'].str.replace(' County', '')
    long_features[i] = long_features[i].melt(id_vars=['County', 'State'], 
            var_name='Date', 
            value_name='Value')
    
    long_features[i]['Year'] = long_features[i].Date.str.split('_', expand=True).iloc[:,-1].astype(int)
    long_features[i] = long_features[i].rename(columns={'Value': long_features[i].Date.str.strip('1234567890_')[1]})
    long_features[i] = long_features[i].drop(['Date'], axis=1)
    print(long_features[i])

(3089, 13)
              County State       inc  Year
0          Abbeville    SC    658355  2009
1      Acadia Parish    LA   1917444  2009
2           Accomack    VA   1092338  2009
3                Ada    ID  14681119  2009
4              Adair    MO    711589  2009
...              ...   ...       ...   ...
33974           Yuma    AZ   7818246  2019
33975           Yuma    CO    492947  2019
33976         Zapata    TX    410289  2019
33977         Zavala    TX    364421  2019
33978        Ziebach    SD     54956  2019

[33979 rows x 4 columns]
(3139, 12)
              County State  inc_ineql  Year
0          Abbeville    SC  17.191542  2010
1      Acadia Parish    LA  14.102745  2010
2           Accomack    VA  17.035170  2010
3                Ada    ID  11.809223  2010
4              Adair    IA  10.648803  2010
...              ...   ...        ...   ...
31385           Yuma    AZ  13.153264  2019
31386           Yuma    CO  11.565673  2019
31387         Zapata    TX  25.272130  2

# Dataset Generation

In [6]:
# merge features to the price dataset as "df"

all_long_features = reduce(lambda left, right: pd.merge(left, right, on=['State','County', 'Year']), long_features)
df = pd.merge(All, all_long_features, how='left', on=['State','County', 'Year'])

In [7]:
df

,Unnamed: 0,County,State,Metro,StateCodeFIPS,MunicipalCodeFIPS,Date,ZHVI,ZHVI_prev,Year,...,CPI,Interest Rate,#private_schools,#public_schools,inc,inc_ineql,BD,highschool,migr,residt
0,0,Anchorage,AK,Anchorage,2,20,2000-02-29,154057.0,153839.0,2000,...,170.0,5.24,14.0,102.0,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Anchorage,AK,Anchorage,2,20,2000-03-31,154234.0,154057.0,2000,...,171.0,5.34,14.0,102.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Anchorage,AK,Anchorage,2,20,2000-04-30,154382.0,154234.0,2000,...,170.9,5.50,14.0,102.0,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Anchorage,AK,Anchorage,2,20,2000-05-31,154670.0,154382.0,2000,...,171.2,5.71,14.0,102.0,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Anchorage,AK,Anchorage,2,20,2000-06-30,154784.0,154670.0,2000,...,172.2,6.00,14.0,102.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
288401,288401,Wyoming,WV,NaN,54,109,2021-05-31,37132.0,37056.0,2021,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
288402,288402,Wyoming,WV,NaN,54,109,2021-06-30,37251.0,37132.0,2021,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
288403,288403,Wyoming,WV,NaN,54,109,2021-07-31,37340.0,37251.0,2021,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
288404,288404,Wyoming,WV,NaN,54,109,2021-08-31,37386.0,37340.0,2021,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
df.isnull().sum()

Unnamed: 0                0
County                    0
State                     0
Metro                 60580
StateCodeFIPS             0
MunicipalCodeFIPS         0
Date                      0
ZHVI                      0
ZHVI_prev                 0
Year                      0
Month                     0
Region Code          134434
UnemploymentRate     134434
#hospital             80763
hospital_rating       80763
CPI                     846
Interest Rate          1952
#private_schools        846
#public_schools         846
inc                  162046
inc_ineql            162046
BD                   162046
highschool           162046
migr                 162046
residt               162046
dtype: int64

In [9]:
df.to_csv('data\All_Adrianne.csv', index = False, header=True)